 <font size="6">Базовые методы Классического Машинного Обучения</font>

# Признаки и работа с ними

>Coming up with features is difficult, time-consuming, requires expert knowledge. "Applied machine" learning is basically feature engineering. @Andrew Ng

Общая схема классического машинного обучения выглядит так. Даже в случае нейросетей некая предобработка исходных данных все равно не бывает лишней.


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/data_preparation.png" width="600">

> Feature engineering is the process of transforming raw data into features that better represent the underlying problem to the predictive models, resulting in improved model accuracy on unseen data. @Dr. Jason Brownlee

**Генерация признаков** - процесс придумывания способов описания данных с помощью простых значений, которые должны отражать характеристики объектов исследований, через которые выражаются целевые значения.

## Данные, требующие предобработки

Изначально объекты в нашем датасете могут быть представлены в виде описаний, которые не являются признаковыми, либо, очевидно, требует некоторой предобработки:

1. веб-страницы 
2. файлы 
3. ссылки на участников группы
4. измерения в разных единицах (см, м, дц)
и т.д 
________
todo вклейка информации

IP адрес: 

xxx.xxx.xxx.xxx -> регион, провайдер

Координаты атомов:

[AlphaFold](https://yakovlev.me/para-slov-za-alphafold2/
)


"как вообще математически можно выразить структуру белка. До того мы постоянно неявно подразумевали, что это координаты атомов (всех, каркасных, Cα или каких-то ещё), но на практике это очень неудачное представление, поскольку оно не единственное. Мы обычно считаем, что предсказание работает как некоторая детерминированная функция: принимает на вход последовательность и всегда возвращает один единственный ответ. Но какой из бесконечного набора координат "канонический"?



"... инвариантом структуры является матрица (таблица) всех попарных расстояний между атомами"

_______

Большая часть моделей неспособна работать с такими представлениями в сыром виде и или просто не запустится, либо будет выдавать неадекватные результаты.

Процесс создания признаков зависит от модели, которую мы собираемся использовать. Для одних моделей полезно добавить признаки, полученные делением/перемножением исходных. Другие модели могут провести эти операции сами и экономнее/менее переобучаясь. Как вариант, добавление признаков, явно зависящих друг от друга, может даже мешать некоторым моделям. 


Например, плохая идея добавлять в обычную линейную модель как признаки X1 и X2, так и их сумму. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/process_of_generation_features.png" width="700">

Вообще говоря, надо понимать, что процесс feature engneering является критическим местом, bottleneck, в машинном обучении. Все, что ваша модель будет знать о данных, решается на этом этапе. Больше, чем вы ей дадите - она не узнает. 

Если вы в данных дадите явную подсказку об ответе - то она будет использовать эту подсказку, а реальные закономерности может и не выучить. К примеру, можно дать ей в качестве признака id покупателя, который каждую неделю покупает одно и то же. Если таких ситуаций будет много, то она и выучит, что надо предсказывать все по id. Когда же к вам придет новый покупатель или у старого, что-то поменяется в поведении, модель начнет вести себя неадекватно. 

Точно такую же роль может сыграть информация о номере эксперимента, лаборатории, в которой его проводили, аспиранте, который его проводил и тд. 

Такая ситуация будет называться **data leakage**.


Ну и понятно, что если вы дадите модели только нерелевантную информацию, она ничего из нее не вытащит. 

> At the end of the day, some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used. @ Prof. Pedro Domingos

> The algorithms we used are very standard for Kagglers. …We spent most of our efforts in feature engineering. … We were also very careful to discard features likely to expose us to the risk of over-fitting our model.  @Xavier Conort, топ-участник Kaggle



## Типы признаков

Традиционно признаки делятся на:




### Вещественные 
Вещественные признаки бывают:

 


* дискретные. Например - число лайков от пользователей
 
 <img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/discrete_features_social_media_likes.jpg" width="1000">

 * непрерывные. Например - температура


 <img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/continuous_features_thermometer.png" width="600">
 


Понятно, что разделение часто условное. Тот же возраст можно посчитать и дискретной переменной (пользователь всегда нам сообщает свои полные года), и непрерывной (возраст можно считать с любой точностью, но никто не будет) )


Также иногда вещественные признаки делят на относительные (считаются относительно чего-то, уже нормированные и тд)  и интервальные. 


### Категориальные 



Значение -  принадлежность к какой-то из категорий. Традиционно делятся на сильно отличающиеся по свойствам:


 * упорядоченные (ординальные) - для каждой пары возможных категорий можем сказать, какая больше, а какая меньше. Например - класс места. Или размер одежды

<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/categorical_ordered_features.png" width="600">



 * неупорядоченные (номинальные) - категории между собой несравнимы. Обычно нельзя сказать, что красный телефон больше синего. Или что солнечная погода больше снежной

<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/categorical_unordered_features.png" width="900">



 


Часто мы сталкиваемся с бинарными категориальными признаками, для которых известно только две возможных категории (например, биологический пол человека).

## Преобразования признаков



### Вещественных признаков 



#### Бинаризация 

Например, нам может быть неинтересно, сколько конкретно раз встретилось явление в наблюдении - главное, что оно вообще встретилось. Тогда мы просто превращаем наш вещественный признак в бинарный "было ли явление", и работаем уже с ним. 



#### Округление

Часто данные до нас доходят с очень высокой точностью после запятой. Нужно ли это нашей модели - часто нет. Иногда по факту два наблюдения не различаются  по этому признаку (разница в пределах статошибки), но по признаку их отличить можно. Это может приводить к переобучению. В таких случаях разумно признаки округлить. 

#### Bining (Бинирование)

Опять же, нам не интересны точные значения - например, что видео набрало 1000 лайков, а не 1001. 

К тому же, число просмотров/лайков некоторых видео может быть очень большим в сравнении с остальными, что будет приводить к неадекватному поведению. 
В итоге часть значений у нас встречается часто, а часть - очень редко. Это может приводить к неадекватному поведению модели. 




##### Fixed-width bining

Просто бьем наши значения по диапазонам фиксированной длины. Так часто поступают с возрастом. 


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/fixed_width_binning.png" width="400">



##### Adaptive Binning

Это не всегда работает хорошо. Например, распределение зарплат у нас очень сильно скошено вправо. 


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/adaptive_binning.png" width="400">



И бинирование с фиксированной длиной бина нам не поможет справиться с редкими значениями.

В этой ситуации помогает бинирование, например, по квантилям - когда границы бина представляют собой квантили. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/quantiles_binning.png" width="400">

#### Логарифмирование

С ситуацией, когда распределение скошено вправо работает и другой подход - прологаримфировать величину. 


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/log_binning.png" width="400">


Обобщением этого подхода является [Box-Cox Transform](https://www.statisticshowto.com/box-cox-transformation/#:~:text=A%20Box%20Cox%20transformation%20is,a%20broader%20number%20of%20tests.), общей целью которой является придать данным вид более похожий на нормальное распределение, с которым работает бОльшее число моделей и сходимость лучше 


### Категориальных признаков 

#### Label encoding 

Просто берем и каждой категории однозначно сопоставляем число. 
Очень простой способ, если признак ординальный - будет работать почти всегда. 

Если же наш признак - номинальный, то могут возникнуть проблемы. Мы не можем сказать, что салатовый больше красного (в большинстве случаев). Но модель ничего про это не знает и после нашего кодирования спокойно такие сравнения может производить. Это может приводить к более низкому качеству модели и выучиванию ею неправильной информации. Кроме того, например, деревьям решений, чтобы выделить в таком случае конкретную категорию придется делать сразу несколько действий, которые, в силу жадности алгоритма их построения, могут и не быть найдены



 Некоторые модели (например, lightgbm) автоматически могут перекодировать все правильно, если им сообщить, что переданный признак - категориальный. Для некоторых это придется делать вручную. 

#### One-hot encoding 

Схемой, которая часто используется на практике, является one-hot encoding. Он состоит том, что вместо одного категориального признака X создается набор бинарных категориальных признаков, которые отвечают на вопрос "X == C? ", где C пробегает все возможные значения категориального признака. 

Теперь чтобы обусловиться на конкретное значение категориального признака, дереву решений достаточно задать один вопрос. 


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/one_hot_encoding.png" width="450">




Однако у такой схемы есть один минус - мы получаем линейно зависимые признаки. Это может плохо влиять на некоторые модели (для случая нейронных сетей - обычно нет, но полезно держать в голове). 


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/problem_of_ohe.png" width="850">

Потому иногда одну из категорий исключают при кодировании, например, в примере выше можно исключить Fish, ведь если все три других признака-категории равны 0, то точно верно, что категория - Fish. 


#### Target encoding 

Кодируем каждую категорию каким-то численным параметром, характеризующим то, что мы предсказываем. Например, можно каждую категорию категориального признака заменять на среднее 

На самом деле, так просто делать нельзя, можно получить переобученную модель. Как делать - можете подробно посмотреть, к примеру, [здесь](https://github.com/Dyakonov/PZAD/blob/master/2020/PZAD2020_042featureengineering_07.pdf) или [здесь](https://www.coursera.org/lecture/competitive-data-science/concept-of-mean-encoding-b5Gxv?redirectTo=%2Flecture%2Fcompetitive-data-science%2Fconcept-of-mean-encoding-b5Gxv) 


#### Embedding

Можно научить вашу модель саму сопоставлять каждой категории некий вектор определенной размерности. Для этого вначале сопоставляем каждой категории случайный вектор заданной длины. А далее изменяем этот вектор как обычные веса. 

<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/embedding.png" width="500">


#### Кодирование циклических категориальных признаков

В случае с такими признаками, как день недели или время суток, мы сталкиваемся с проблемой того, что нам нужно предложить кодирование, которое будет учитывать, что понедельник близок к воскресенью так же, как понедельник же ко вторнику, и тд. 

В случае деревьев решений и методов на них основанных можно "забить" - такие методы сами разберутся. Для некоторых других методов, тех же нейросетей, правильно кодирование может улучшить качество и сходимость. 



Давайте нанесем наши категории, например, дни недели - на окружность. Как это сделать? 
Пусть понедельнику соответствует 1, а воскресенью - 7. Далее посчитаем два таких вспомогательных признака по следующим формулам

In [ ]:
import numpy as np
weekdays = np.arange(1, 8) #create an array of weekdays
print(weekdays)
sina = np.sin(weekdays * np.pi * 2 / np.max(weekdays)) #feature 1
cosa = np.cos(weekdays * np.pi * 2 / np.max(weekdays)) #feature 2

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(7,7)) #Decide figure size
plt.scatter(sina, cosa) #Plot scatter of feature 1 vs feature 2
for  i, z in enumerate( ("Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun") ): #for each day in a week
  plt.text(sina[i], cosa[i], s=z) #add text labels to plot
  

Что делать дальше? А по сути, мы уже все сделали. Теперь расстояния между понедельником и вторником и воскресеньем и понедельником одинаковые:

In [ ]:
dist_mon_tue = (sina[1] - sina[0]) ** 2 + (cosa[1] - cosa[0]) ** 2 #distance between Monday and Tuesday
dist_sun_mon = (sina[6] - sina[0]) ** 2 + (cosa[6] - cosa[0]) ** 2 #distance between Sunday and Monday
print('Distance between Mon-Tue = %.2f' % dist_mon_tue)
print('Distance between Sun-Mon = %.2f' % dist_sun_mon)

то же самое верно и для любых отстоящих друг от друга на одинаковое число дней



In [ ]:
dist_mon_wed = (sina[2] - sina[0]) ** 2 + (cosa[2] - cosa[0]) ** 2 #distance between Monday and Wednesday
dist_fri_sun = (sina[4] - sina[6]) ** 2 + (cosa[4] - cosa[6]) ** 2 #distance between Friday and Sunday
print('Distance between Mon-Wed = %.2f' % dist_mon_wed)
print('Distance between Fri-Sun = %.2f' % dist_fri_sun)

Таким образом, циклические признаки можно кодировать парой признаков - sin и cos, полученных по схеме, описанной выше. 



Проблемы подхода

1. Деревья решений могут решить задачу и так. А такое кодирование им, наоборот, будет мешать, т.к. они работают с одним признаком за раз

2. Надо понимать, что важность исходной категориальной фичи неочевидным образом делится между двумя полученными из нее таким образом. 

3. В некоторых задачах one-hot работает лучше

### Кодирование взаимодействия признаков

Признаки могут по-разному взаимодейстовать и некоторые модели в принципе не могут моделировать это взаимодействие. 


Взаимодействовать могут вещественные переменные и категориальные


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/strength_vs_speed.png" width="550">

**Сила vs. скорость**

категориальные и категориальные 


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/categorical_and_categorical.png" width="300">

вещественные и вещественные 


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/real_and_real.png" width="300">


Могут быть и более высокоуровневые взаимодействия - взаимодействуют много разных признаков.

Взаимодействия могут быть самые разные - много способов кодировать. Например, добавлять в число признаков их произведение. 

## Генерация признаков

Если у вас есть модель, обученная на другом датасете, можно генерировать признаки при помощи нее. Например, при помощи случайного леса


<img src ="https://edunet.kea.su/repo/EduNet-content/L04/img_license/generating_features_using_model.png" width="700">

**Генерация бинарного признакового пространства с помощью RandomForest**

## Нормализация данных

Загрузим датасет с образцами здоровой и раковой ткани. Датасет состоит из 569 примеров, где каждой строчке из 30 признаков, соответствует класс `1` злокачественной (*malignant*) или `0` доброкачественной (*benign*) ткани. Задача состоит в том, чтобы по 30 признакам обучить модель определять тип ткани (злокачественная или доброкачественная).

Можно иметь сколь угодно хороший алгоритм для классификации - но до тех пор, пока данные на входе - мусор, на выходе из нашего чудесного классификатора мы тоже будем получать мусор (*garbage in - garbage out*). Давайте разберемся, что конкретно надо сделать, чтобы kNN реально заработал.


In [ ]:
import sklearn.datasets

cancer = sklearn.datasets.load_breast_cancer() # load data
X = cancer.data # features
Y = cancer.target # labels(classes)
print(f'X shape: {X.shape}, Y shape: {Y.shape}') 
print(f'X[0]: \n {X[0]}') 
print(f'Y[0]: \n {Y[0]}') 

Посмотрим сколько данных в классе `0` и сколько данных в классе `1`

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5)) # set fig size 
plt.bar(1,Y[Y==1].shape, label=cancer.target_names[0]) # 1 label 
plt.bar(0,Y[Y==0].shape, label=cancer.target_names[1]) # 0 label
plt.title('Class balance') 
plt.ylabel('Num examples') 
plt.xticks(ticks=[1,0], labels=['1','0']) 
plt.legend(loc='upper left') 
plt.show() 

Теперь давайте посмотрим на сами данные. У нас есть 569 строк в каждой из которой по 30 колонок. Такие колонки называют признаками или *features*. Попробуем математически описать все эти признаки (mean, std, min и тд)

In [ ]:
import pandas as pd
pd.DataFrame(X).describe()

То же самое, но в виде графика. Видно, что у фич совершенно разные значения.

In [ ]:
import seaborn as sns

ax = sns.boxenplot(data=pd.DataFrame(X), orient="h", palette="Set2")
ax.set(xscale='log', xlim=(1e-4, 1e4), xlabel='Values', ylabel='Features')
plt.show()

Чтобы адекватно сравнить данные между собой нам следует использовать нормализацию.

 **Нормализация, выбор Scaler**

Нормализация — это преобразование данных к неким безразмерным единицам.
Ключевая цель нормализации — приведение различных данных в самых разных единицах измерения и диапазонах значений к единому виду, который позволит сравнивать их между собой.

Главное условие правильной нормализации — все признаки должны быть равны в возможностях своего влияния.

Например, у нас есть данные по группе людей: *возраст* (в годах) и *размер дохода* (в рублях). Возраст может измениться в диапазоне от 18 до 70 ( интервал 70-18 = 52). А доход от 30 000 р до 500 000 р (интервал 500 000 - 30 000 = 470 000). В таком варианте разница в возрасте имеет меньшее влияние, чем разница в доходе. Получается, что доход становится более важным признаком, изменения в котором влияют больше при сравнении схожести двух людей.

Должно быть так, чтобы максимальные изменения любого признака в «основной массе объектов» были одинаковы. Тогда потенциально все признаки будут равноценны.

Осталось определиться с выбором инструмента, часто используют следующие варианты: `MinMaxScaler`, `StandardScaler`, `RobustScaler`

Сравним `MinMaxScaler`, `StandardScaler`, `RobustScaler` для признака `data[:,0]`. **Обратите внимание на ось X**

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import numpy as np

np.random.seed(42)  # setting the initialization parameter for random values

# generate random values from 1 to 255, shape (30,1)
test = X[:,0].reshape(-1,1)

plt.figure(1, figsize=(30, 5))  
plt.subplot(141)  # set location
plt.scatter(test, range(len(test)), c=Y)  
plt.ylabel("Num examples", fontsize=15)  
plt.xticks(fontsize=15)  
plt.yticks(fontsize=15)  
plt.title("Non scaled data", fontsize=18)  

# scale data with MinMaxScaler
test_scaled = MinMaxScaler().fit_transform(test)  
plt.subplot(142)
plt.scatter(test_scaled, range(len(test)), c=Y)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("MinMaxScaler", fontsize=18)

# scale data  with StandardScaler
test_scaled = StandardScaler().fit_transform(test)  
plt.subplot(143)
plt.scatter(test_scaled, range(len(test)), c=Y)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("StandardScaler", fontsize=18)

# scale data  with RobustScaler
test_scaled = RobustScaler().fit_transform(test)  
plt.subplot(144)
plt.scatter(test_scaled, range(len(test)), c=Y)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("RobustScaler", fontsize=18)
plt.show()

Идея **`MinMaxScaler`** заключается в том, что он преобразует данные в диапазоне от 0 до 1. Может быть полезно, если нужно выполнить преобразование, в котором отрицательные значения не допускаются (e.g., масштабирование RGB пикселей)


$$z=\frac{X_i-X_{min}}{X_{max}-X_{min}}$$

$X_{min}$ и $X_{max}$ задаются как минимальное и максимальное допустимое значение, по умолчанию:  $X_{min}=0$  и $X_{max}=1$

Идея **`StandardScaler`** заключается в том, что он преобразует данные таким образом, что распределение будет иметь среднее значение 0 и стандартное отклонение 1. Большинство значений будет в  диапозоне от -1 до 1. Это стандартная трансформация, и она применима во многих ситуациях.

$$z=\frac{X-u}{s}$$

$u$ — среднее значение (или 0 при `with_mean=False`) и $s$ — стандартное отклонение (или 0 при `with_std=False`)

И StandardScaler и MinMaxScaler очень чувствительны к наличию выбросов. **`RobustScaler`** использует медиану и основан на *процентилях*. Процентиль — мера, в которой процентное значение общих значений равно этой мере или меньше ее. Например, 90 % значений данных находятся ниже 90-го процентиля, а 10 % значений данных находятся ниже 10-го процентиля. Соответственно, RobustScaler не зависит от небольшого числа очень больших предельных выбросов (outliers). Следовательно, результирующий диапазон преобразованных значений признаков больше, чем для предыдущих скэйлеров и, что более важно, примерно одинаков.

$$z=\frac{X-X_{median}}{IQR}$$

$X_{median}$ — значение медианы, $IQR$ — межквартильный диапазон равный разнице между 75-ым и 25-ым процентилями

Для нашей задачи по определению раковых опухолей обработаем наши 30 признаков с помощью StandardScaler.

In [ ]:
X_norm = StandardScaler().fit_transform(X)  # scaled data

Видим что они стали намного более сравнимы между собой.

In [ ]:
pd.DataFrame(X_norm).describe()

In [ ]:
ax = sns.boxenplot(data=pd.DataFrame(X_norm), 
                   orient="h", 
                   palette="Set2")
ax.set(xlabel='Values', ylabel='Features')
plt.show()

## Дополнительные источники по работе с признаками

1. [Feature Selection for High-Dimensional Data](https://www.springer.com/gp/book/9783319218571)
2. [How to Win a Data Science Competition: Learn from Top Kagglers](https://www.coursera.org/learn/competitive-data-science)
3. **Feature Engineering for Machine Learning: Principles and Techniques for Data Scientists Paperback** – April 14, 2018 by Alice Zheng , Amanda Casar
4. [Сайт](https://dyakonov.org/) и [курс](https://github.com/Dyakonov/PZAD) Дьяконова
5. Серия статей на towardsdatascience, [первая из серии](https://towardsdatascience.com/understanding-feature-engineering-part-1-continuous-numeric-data-da4e47099a7b)
6. [A Few Useful Things to Know About Machine Learning](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)
7. [Про кодирование циклических признаков](http://blog.davidkaleko.com/feature-engineering-cyclical-features.html)

#Разделение train-validation-test 

## ??? Проблема переобучения 
idea: дать студентам визуальное представление о том, что такое переобучение.

todo: Опраксимация линейной зашумленной функции полиномом n-ой степени. Показать ошибку на трейне и на тесте. Гипперпараметр - степень полинома https://scikit-learn.org/stable/auto_examples/linear_model/plot_polynomial_interpolation.html

Теперь обучим kNN для общей выборки данных, при разном значении количества соседей.

In [ ]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


np.random.seed(42)  # setting the initialization parameter for random values

n_nei_rng = np.arange(1, 31)  # array of the number of neighbors

quality = np.zeros(
    n_nei_rng.shape[0]
)

X_norm = StandardScaler().fit_transform(X)  # scaled data

for ind in range(n_nei_rng.shape[0]):  # for all elements
    # create knn for all num neighbors 
    knn = KNeighborsClassifier(
        n_neighbors=n_nei_rng[ind]
    )  
    knn.fit(X_norm, Y)  
    q = accuracy_score(y_pred=knn.predict(X_norm), y_true=Y)  # accuracy
    quality[ind] = q  # fill quality

plt.figure(figsize=(8, 5))  
plt.title("KNN on train", size=20)  
plt.xlabel("Neighbors", size=15)  
plt.ylabel("Accuracy", size=15)  
plt.plot(n_nei_rng, quality)  
plt.xticks(n_nei_rng) 
plt.show()  

Видим, что качество на 1 соседе - самое лучшее. Но это и понятно - ближайшим соседом элемента из обучающей выборки будет сам объект. Мы просто **запомнили** все объекты.

Если теперь мы попробуем взять какой-то новый образец опухоли и классифицировать его - у нас скорее всего ничего не получится. В таких случаях мы говорим, что наша модель не умеет обобщать (*generalization*).

Для того, чтобы знать заранее обобщает ли наша модель или нет, мы можем разбить все имеющиеся у нас данныe на 2 части. Но одной части мы будем обучать классификатор (*train set*), а на другой тестировать насколько хорошо он работает (*test set*).

In [ ]:
from sklearn.model_selection import train_test_split

# split data to train/test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)

scaler = StandardScaler()  
scaler.fit(X_train)  
X_train_norm = scaler.transform(X_train)  # scaling data
X_test_norm = scaler.transform(X_test)  # scaling data

n_nei_rng = np.arange(1, 31)  
train_quality = np.zeros(n_nei_rng.shape[0])  # quality on train data
test_quality = np.zeros(n_nei_rng.shape[0])  # quality on test data

for ind in range(n_nei_rng.shape[0]):  
    knn = KNeighborsClassifier(n_neighbors=n_nei_rng[ind])  
    knn.fit(X_train_norm, Y_train)  
    
    # accuracy on train data
    trq = accuracy_score(y_pred=knn.predict(X_train_norm), y_true=Y_train)  
    train_quality[ind] = trq  

    # accuracy on test data
    teq = accuracy_score(y_pred=knn.predict(X_test_norm), y_true=Y_test)  
    test_quality[ind] = teq  

# accuracy plot  on train and test data
plt.figure(figsize=(8, 5))
plt.title("KNN on train vs test", size=20)
plt.plot(n_nei_rng, train_quality, label="train")
plt.plot(n_nei_rng, test_quality, label="test")
plt.legend()
plt.xticks(n_nei_rng)
plt.xlabel("Neighbors", size=15)
plt.ylabel("Accuracy", size=15)
plt.show()

Вот, теперь мы видим, что 1 сосед был "ложной тревогой". Такие случаи мы называем *переобучением*. Чтобы действительно предсказывать что-то полезное нам надо выбирать число соседей начиная минимум с 3

##Проблема подбора гиперпараметров на тестовой выборке

Каждая модель имеет как ряд **параметров**, которые она меняет в процессе обучения (например, веса модели), так и ряд **гиперпараметров**, которые влияют на то, каким способом модель меняет параметры в процессе обучения. 

В случае kNN параметры, строго говоря, отсутствует - модель просто запоминает объекты обучающей выборки. Особо упорные могут считать их параметрами. 

А вот гиперпараметры есть, даже несколько групп. Какие? 

1. число соседей 
2. функция, которой считаем расстояние между объектами (L2=eucledian, L1=manhattan)
3. веса, с которыми складываем метки ближайших соседей
4. признаки! (но об этом с вами поговорим позже)
5. сама модель - мы могли выбрать не kNN, а нагуглить что-нибудь другое

Посмотрим еще раз на график, который нарисовали на прошлом шаге. Какое число соседей считать оптимальным? Метрика явно скачет? 

In [ ]:
plt.figure(figsize=(8, 5))
plt.title("KNN on train vs test", size=20)
plt.plot(n_nei_rng, train_quality, label="train")
plt.plot(n_nei_rng, test_quality, label="test")
plt.xlabel("Neighbors", size=15)
plt.ylabel("Accuracy", size=15)
plt.legend()
plt.xticks(n_nei_rng)
plt.show()


Не понятно, насколько результат зависит от того, как нам повезло или не повезло с разбиением данных на обучение и тест. Может оказаться так, что для конкретного разбиения хорошо выбрать k=5, а для другого - k=7. 

Кроме того, опять же - фактически, мы сами выступаем в роли модели, которая учит гиперпараметры (а не параметры) под видимую ей выборку. 

Представим себе, что у нас есть 10000 моделей, полученных подкручиванием разных гиперпараметров (в том числе, выбором просто разного типа модели). Представим, что все эти модели не работают. Вообще. Представим так же, что каждая модель угадывает класс в задаче разделения на два класса с вероятностью 0.5 (и будем считать, что классы у нас сбалансированны - то есть 50% одного класса и 50% другого). 
Опять же, понятно, что классификация такими моделями ничем не лучше подбрасывания монетки. 



In [ ]:
def guess_model(Y_real):
    # array with values True/False, for random choice by mask 
    guessed = np.random.choice(
        [True,False],  
        size=Y_real.shape[0],
        replace=True,
    )
    Y_pred = np.zeros_like(Y_real)  # zeros array, shape Y_real
    
    # with mask 'guessed' assign values 
    Y_pred[guessed] = Y_real[guessed]  
    Y_pred[~guessed] = 1 - Y_real[~guessed]  
    return Y_pred

In [ ]:
models_num = 10000  # num of experiments
best_quality = 0.5  # порог качества по точности

# array with random values in range 0, 1
Y_real = np.random.choice(
    [0, 1], size=250, replace=True
)  

for i in range(models_num):  # for all expirements
    Y_pred = guess_model(Y_real)  # predicted values
    q = accuracy_score(y_pred=Y_pred, y_true=Y_real)  # accuracy
    if q > best_quality:  
        best_quality = q   
print(f"Best result {best_quality}")

То есть мы перебором всех возможных моделей вполне можем получить для абсолютно бесполезной модели приемлемое качество


Получается, что если подбирать гиперпараметры модели на *train set*, то:
1. можно переобучитьcя, просто на более "высоком" уровне. Особенно если гиперпараметров у модели много и все они разнообразны
2. нельзя быть уверенным, что выбор параметров не зависит от разбиения на обучение и тест 

Поэтому мы:

1. подбираем гиперпараметры моделей на отдельном датасете, называмым валидационным. Получаем мы его разбиением обучающего датасета на собственно обучающий и валидационный 

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/split_dataset_for_train_val_test.png" width="700">




## Примеры ошибок в данных и при разбиении

todo Сюда можно добавить истоию с Хаски 

Еще одним преимуществом наличия валидационного набора является то, что вы можете сделать **раннюю остановку** (*early stopping*), когда во время обучения одной модели, модель оценивается по валидационному набору на каждой итерации процесса обучения. Обучение прекращается, когда результат валидации начинает снижаться, так как это указывает на то, что модель начинает переобучаться на *train set*.

В процессе обучения модели, мы вообще-то хотели бы знать насколько хорошо она справляется с поставленной задачей. Если для оценки модели на каждой итерации (работы над моделью) мы будем использовать тестовый сет, мы просто оптимизируем модель к этому конкретному (тестовому) разбиению и потерям возможность объективно оценивать обобщающую способность модели (тестовое множество станет неявной частью процесса обучения [Cawley, 2010](https://www.jmlr.org/papers/volume11/cawley10a/cawley10a.pdf)). Вместо этого для измерения качества модели (и "руководством над процессом обучения") мы можем использовать отдельный (третий) набор для проверки - *validation/val set*.


### Validation set

В следующий раз вам стоит воспользоваться тестом только уже при проверке обученной модели. А вот валидационный (*validation set*) нам понадобиться при обучении модели. 

In [ ]:
plt.scatter(X_train[:,0], X_train[:,1], color='red', alpha=0.5, label='Train')
plt.scatter(X_val[:,0], X_val[:,1], color='green', alpha=0.5, label='Val')
plt.scatter(X_test[:,0], X_test[:,1], color='blue', alpha=0.5, label='Test')
plt.legend()

Вспомним "Таблицу умножения". Если мы хотим проверить умение умножать, то проверки примерами из таблицы умножения будет недостаточно, ведь она может быть полностью запомнена. Нужно давать новые примеры, которых не было в таблице умножения (обучающей выборке).

Если модель "запомнит всё", то она будет идеально работать на данных, которые мы ей показали, но может вообще не работать на любых других данных.

С практической точки зрения важно, как модель будет вести себя именно на незнакомых для неё данных. То есть насколько хорошо она научилась обобщать закономерности, которые в данных присутствовали (если они вообще существуют).

Для оценки этой способности набор данных разделяют на две, а иногда даже на три части:

* train — Данные, на которых модель учится;
* validation/test — Данные, на которых идет проверка.

В `sklearn.model_selection` есть модель для разделения массива данных на тренировочную и тестовую часть. 

In [ ]:
# Split data
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(features, class_labels, test_size=0.2) # 80% training and 20% test

print("X_train shape",X_train.shape)
print("X_test shape",X_test.shape)

А мы в дальнейшем будем пользоваться аналогичными инструментами библиотеки PyTorch.

info: L1-Intro

Разбиение данных на обучающую и тестовую выборки 

Самым простым способом научиться чему-либо является "запомнить всё".

## Используйте соответствующий тестовый датасет(из publish or perish)

Прежде всего, всегда используйте тестовый набор для измерения обобщающей способности модели ML. То, насколько хорошо модель работает на обучающем множестве (*train set*), на практике не имеет никакого значения, и достаточно сложная модель может на нем полностью переобучится, но так и не научится обобщать тестовые данные. 


#####❗**КАРТИНКА С ПЕРЕОБУЧЕНИЕМ (Расхождение train и test)**

Аналогичные ситуации могут возникать, когда для сбора обучающих и тестовых данных используется разное оборудование. **Например**, для обучения вы используете томограф из одного госпиталя, а оценку качества модели проводите на томографе из другого госпиталя. Если модель не учитывает характеристики оборудования, она, скорее всего, не будет обобщаться на томограф из второй больницы (при этом на своем собственном датасете из первого госпиталя, эту ошибку вы отловить не сможете).

#####❗ **МЕСТО ДЛЯ КАРТИНКИ С ФОТОГРАФИЯМИ**

Также важно убедиться, что данные в тестовом наборе на самом деле подходят. То есть, они не должны пересекаться с обучающим набором и в то же время в достаточной степени отражать реальные данные. **Например**, рассмотрим датасет фотографий объектов, где изображения в обучающем и тестовом наборе были получены на открытом воздухе в солнечный день. Наличие одинаковых погодных условий означает, что тестовое множество не будет независимым, а поскольку оно не охватывает более широкий спектр погодных условий, оно также не будет репрезентативным.

## Правильно делайте разбиения (из Publish or perish) 

Что бы обучить модель, нам понадобиться разбить свои данные на несколько частей (`train-val-test`).


### Рассмотрим пример

Давайте создадим датасет

In [ ]:
import numpy as np

dataset_size = 1000
n_features = 2

# Создадим рандомный датасет
X = np.random.normal(size=(dataset_size,n_features))
y = np.random.normal(size=(dataset_size,)) 
print('Размерность X', X.shape)
print('Размерность y', y.shape)

И разделим его на `train`, `val` и `test`. Если вы применяете случайное разбиение (как в следующем блоке кода), не забудьте зафиксировать random state, что бы в следующий раз вы получили ровно такую же тестовую выборку.

In [ ]:
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15, random_state=42)

print('Размерность X_train', X_train.shape)
print('Размерность X_val', X_val.shape)
print('Размерность X_test', X_test.shape)

## Оптимизируйте гиперпараметры вашей модели (Publish or perish)

Многие модели имеют **гиперпараметры** - то есть числа или параметры, которые влияют на конфигурацию модели. Примерами могут служить: функция ядра, используемая в SVM; количество деревьев в случайном лесу и архитектура нейронной сети. Многие из этих гиперпараметров существенно влияют на производительность модели, и, как правило, универсальных гиперпараметров (таких, которые были бы оптимальны для всех возможных задач) не существует.

То есть, чтобы получить максимальную отдачу от модели, гииперпараметры нужно подбирать под конкретный набор данных. Хотя может возникнуть соблазн возиться с гиперпараметрами до тех пор, пока вы не найдете что-то подходящее, такой подход, скорее всего, не будет оптимальным. Гораздо лучше использовать какую-то стратегию **оптимизации гиперпараметров** (в качестве бонуса, обоснованная стратегия, в публикации смотрится значительно лучше, чем что-то в стиле *hyperparameters were chosen by chance*). Базовые стратегии включают случайный поиск и поиск по сетке, но они не очень хорошо масштабируются для большого количества гиперпараметров или для моделей, которые дорого обучать, поэтому стоит использовать инструменты, которые ищут оптимальные конфигурации более интеллектуальным способом (исследовано в [Yang et al., 2020](https://arxiv.org/abs/2007.15745)).

#### Библиотеки для оптимизации гиперпараметров

Существует довольно много библиотек для оптимизации гиперпараметров. Ключевыми являются [Ray-tune](https://docs.ray.io/en/latest/tune/index.html), [Optuna](https://optuna.readthedocs.io/en/stable/) и [Hyperopt](https://github.com/hyperopt/hyperopt). В целом они друг от друга принципиально не отличаются, так что скорее это вопрос вкуса. В качестве примера (и для разнообразия) рассмотрим библиотеку `Ray-Tune`.

##### **Ray-Tune**

In [ ]:
!pip install ray[tune] tune-sklearn

In [ ]:
from ray.tune.sklearn import TuneGridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()
X = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 14)

# Example parameters to tune from SGDClassifier
parameter_grid = {"alpha": [1e-4, 1e-1, 1], "epsilon": [0.01, 0.1]}

tune_search = TuneGridSearchCV(
    SGDClassifier(),
    parameter_grid,
    early_stopping=True,
    max_iters=10)

tune_search.fit(x_train, y_train)

#best set of perameter
print(tune_search.best_params_)

#best score with best set of perameters
print(tune_search.best_score)

Запомните, когда вы оптимизируете гиперпараметры или признаки, используемые моделью, для их отбора нужно использовать отдельный, валидационный датасет, или же кросс-валидацию. Вы **НЕ должны подбирать модель или ее гиперпараметры на тесте**.

Также можно использовать методы **AutoML** для оптимизации выбора модели и ее гиперпараметров (см. обзор в [He et al., 2021](https://arxiv.org/abs/1908.00709)).

#Кросс-валидация






### Перекрестная валидация (*cross-validation*) Publish or perish

Если вы не можете получить больше данных - то вы можете лучше использовать уже имеющиеся, используя перекрестную валидацию (*cross validation*). 


Вообще говоря, кросс-валидация используется для более точной оценки модели. Одна оценка модели может быть ненадежной и может либо недооценить, либо переоценить истинный потенциал модели. По этой причине обычно проводится несколько оценок. Существует несколько способов провести множественную оценку модели, и большинство из них предполагает многократное обучение модели с использованием различных подмножеств обучающих данных. **Перекрсная валидация** (CV) особенно популярна и имеет множество разновидностей [Arlot et al., 2010](https://projecteuclid.org/journals/statistics-surveys/volume-4/issue-none/A-survey-of-cross-validation-procedures-for-model-selection/10.1214/09-SS054.full).

**Перекрсная валидация** в машинном обучении подразумеват, что вместо того, чтобы разделить наш набор данных на две части, одну для обучения (`train`), а другую для тестирования (`test`), мы разбиваем наш набор данных на несколько частей, обучаем на некоторых из них, а остальные используем для тестирования. Затем мы используем другие части для обучения и тестирования нашей модели. Это гарантирует, что наша модель обучается и тестируется на новых данных на каждом новом шаге.

Чаще всего используют десятикратную прекресную валидацию.

#### Разберем на примере

Создадим датасет

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=1000, noise=0.3, random_state=42)

plt.scatter(X[:,0], X[:,1], c=y)

И разделим его на 10 (*K*) складок (*folds*)

In [ ]:


classifier = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
X, y = make_moons(noise=0.3, random_state=42, n_samples=1000)

scores = []

# Lets test 100 times
for i in range(0,100):
    # For each experiment lets choose a different random state for splitting
    random_state = np.random.randint(0,2**32)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.4, random_state=random_state
    )

    # normalize data
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    # predict using each classifier
    classifier.fit(X_train, y_train)
    scores.append([random_state, classifier.score(X_test, y_test)])

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

#Split data intto 10 folds
kf = KFold(n_splits=10)
kf.get_n_splits(X)

#Define scaler and classifier
scaler = MinMaxScaler()
clf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)

#Define figure space
fig,ax = plt.subplots(nrows=2, ncols=5, figsize=(10,4))

row = 0
scores = []

#Itterate over folds
for col, (train_index, test_index) in enumerate(kf.split(X)):
    #Split
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    #Normalize
    scaler.fit(X_train)
    scaler.fit(X_test)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    #Classify
    clf.fit(X_train, y_train)

    #Gauge performance
    score = clf.score(X_test, y_test)
    scores.append(score)

    #Plot figure
    if col > 4:
        col-=5
        row=1

    ax[row, col].scatter(X_train[:,0], X_train[:,1], c=y_train, alpha=0.05)
    ax[row, col].scatter(X_test[:,0], X_test[:,1], c=y_test, marker='x')
    ax[row, col].set_title(score)
    ax[row, col].axis('off')

Результирующую точность модели мы определим как среднее от всеех складок

In [ ]:
print('Финальная точность = %.2f ± %.2f)' % (np.mean(scores), np.std(scores)))

Для б**о**льшей надежности можно использовать метод **вложенной перекрестной валидации** (также известный как двойная кросс-валидация, (см. [Cawley, 2010](https://www.jmlr.org/papers/volume11/cawley10a/cawley10a.pdf) и [Wainer et al., 2021](https://research-portal.uea.ac.uk/en/publications/nested-cross-validation-when-selecting-classifiers-is-overzealous)).

### Кросс-валидация (L02)

Каждая модель имеет как ряд **параметров**, которые она меняет в процессе обучения (например, веса модели), так и ряд **гиперпараметров**, которые влияют на то, каким способом модель меняет параметры в процессе обучения. 

В случае kNN параметры, строго говоря, отсутствует - модель просто запоминает объекты обучающей выборки. Особо упорные могут считать их параметрами. 

А вот гиперпараметры есть, даже несколько групп. Какие? 

1. число соседей 
2. функция, которой считаем расстояние между объектами (L2=eucledian, L1=manhattan)
3. веса, с которыми складываем метки ближайших соседей
4. признаки! (но об этом с вами поговорим позже)
5. сама модель - мы могли выбрать не kNN, а нагуглить что-нибудь другое

Посмотрим еще раз на график, который нарисовали на прошлом шаге. Какое число соседей считать оптимальным? Метрика явно скачет? 

In [ ]:
plt.figure(figsize=(8, 5))
plt.title("KNN on train vs test", size=20)
plt.plot(n_nei_rng, train_quality, label="train")
plt.plot(n_nei_rng, test_quality, label="test")
plt.xlabel("Neighbors", size=15)
plt.ylabel("Accuracy", size=15)
plt.legend()
plt.xticks(n_nei_rng)
plt.show()


Не понятно, насколько результат зависит от того, как нам повезло или не повезло с разбиением данных на обучение и тест. Может оказаться так, что для конкретного разбиения хорошо выбрать k=5, а для другого - k=7. 

Кроме того, опять же - фактически, мы сами выступаем в роли модели, которая учит гиперпараметры (а не параметры) под видимую ей выборку. 

Представим себе, что у нас есть 10000 моделей, полученных подкручиванием разных гиперпараметров (в том числе, выбором просто разного типа модели). Представим, что все эти модели не работают. Вообще. Представим так же, что каждая модель угадывает класс в задаче разделения на два класса с вероятностью 0.5 (и будем считать, что классы у нас сбалансированны - то есть 50% одного класса и 50% другого). 
Опять же, понятно, что классификация такими моделями ничем не лучше подбрасывания монетки. 



In [ ]:
def guess_model(Y_real):
    # array with values True/False, for random choice by mask 
    guessed = np.random.choice(
        [True,False],  
        size=Y_real.shape[0],
        replace=True,
    )
    Y_pred = np.zeros_like(Y_real)  # zeros array, shape Y_real
    
    # with mask 'guessed' assign values 
    Y_pred[guessed] = Y_real[guessed]  
    Y_pred[~guessed] = 1 - Y_real[~guessed]  
    return Y_pred

In [ ]:
models_num = 10000  # num of experiments
best_quality = 0.5  # порог качества по точности

# array with random values in range 0, 1
Y_real = np.random.choice(
    [0, 1], size=250, replace=True
)  

for i in range(models_num):  # for all expirements
    Y_pred = guess_model(Y_real)  # predicted values
    q = accuracy_score(y_pred=Y_pred, y_true=Y_real)  # accuracy
    if q > best_quality:  
        best_quality = q   
print(f"Best result {best_quality}")

То есть мы перебором всех возможных моделей вполне можем получить для абсолютно бесполезной модели приемлемое качество


Получается, что если подбирать гиперпараметры модели на *train set*, то:
1. можно переобучитьcя, просто на более "высоком" уровне. Особенно если гиперпараметров у модели много и все они разнообразны
2. нельзя быть уверенным, что выбор параметров не зависит от разбиения на обучение и тест 

Поэтому мы:

1. подбираем гиперпараметры моделей на отдельном датасете, называмым валидационным. Получаем мы его разбиением обучающего датасета на собственно обучающий и валидационный 

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/split_dataset_for_train_val_test.png" width="700">

2. чаще всего делаем несколько таких разбиений по какой-то схеме, чтобы получить уверенность оценок качества для моделей с разными гиперпараметрами - **кросс-валидация**

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/cross_validation_on_train_data.png" width="500">

Часто применяется следующий подход, называемый [K-Fold кросс-валидацией](https://scikit-learn.org/stable/modules/cross_validation.html):

Берется тренировочная часть датасета, разбивается на части - блоки. Дальше мы будем использовать для проверки первую часть (Fold 1), а на остальных учиться. И так последовательно для всех частей. В результате у нас будут информация о точности для разных фрагментов данных и уже на основании этого можно понять, насколько значение этого параметра, который мы проверяем, зависит или не зависит от данных. То есть если у нас от разбиения точность при одном и том же К меняться не будет, значит мы подобрали правильное К. Если она будет сильно меняться в зависимости от того, на каком куске данных мы проводим тестирование, значит, надо попробовать другое К и если ни при каком не получилось - то это такие данные.  


Подберем параметры для модели с помощью **GridSearchCV**.

GridSearchCV – это инструмент для автоматического подбирания параметров для моделей машинного обучения. GridSearchCV находит наилучшие параметры, путем обычного перебора: он создает модель для каждой возможной комбинации параметров

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from IPython.display import clear_output
"""
Parameters for GridSearchCV:
estimator — model
cv — num of fold to cross-validation splitting 
param_grid — parameters names
scoring — metrics 
n_jobs - number of jobs to run in parallel, -1 means using all processors.
"""
model = GridSearchCV(
    estimator=KNeighborsClassifier(),
    cv=KFold(3, shuffle=True, random_state=42),
    param_grid={
        "n_neighbors": np.arange(1, 31),
        "metric": ["euclidean", "manhattan"],
        "weights": ["uniform", "distance"],
    },
    scoring="accuracy",
    n_jobs=-1,
)
model.fit(X_train_norm, Y_train)
clear_output()

Выведем лучшие гиперпараметры для модели, которые подобрали:

In [ ]:
print("Metric:", model.best_params_["metric"])
print("Num neighbors:", model.best_params_["n_neighbors"])
print("Weigths:", model.best_params_["weights"])


Объект GridSearchCV можно использовать как обычную модель

In [ ]:
from sklearn.metrics import balanced_accuracy_score

Y_pred = model.predict(X_test_norm)
print(f"Percent correct predictions {np.round(accuracy_score(y_pred=Y_pred, y_true=Y_test)*100,2)} %")
print(f"Percent correct predictions(balanced classes) {np.round(balanced_accuracy_score(y_pred=Y_pred, y_true=Y_test)*100,2)} %")

Мы можем извлечь дополнительные данные о кроссвалидации и по ключу обратиться к результатам всех моделей

In [ ]:
list(model.cv_results_.keys())

Выведем для примера mean_test_score:

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(model.cv_results_["mean_test_score"])
plt.title("mean_test_score", size=20)
plt.xlabel("Num of experiment", size=15)
plt.ylabel("Accuracy", size=15)
plt.show()

Построим, например, при фиксированных остальных параметрах (равных лучшим параметрам), качество модели на валидации в зависимости от числа соседей

In [ ]:
selected_means = []
selected_std = []
n_nei = []
for ind, params in enumerate(model.cv_results_["params"]):
    if (
        params["metric"] == model.best_params_["metric"]
        and params["weights"] == model.best_params_["weights"]
    ):
        n_nei.append(params["n_neighbors"])
        selected_means.append(model.cv_results_["mean_test_score"][ind])
        selected_std.append(model.cv_results_["std_test_score"][ind])

Построим error bar, для сравнения разброса ошибки при разном количестве соседей Neighbors

In [ ]:
plt.figure(figsize=(8, 5))
plt.title(f"KNN CV, {params['metric']}, {params['weights']}", size=20)
plt.errorbar(n_nei, selected_means, yerr=selected_std, linestyle="None", fmt="-o")
plt.xticks(n_nei)
plt.ylabel("Mean_test_score", size=15)
plt.xlabel("Neighbors", size=15)

plt.show()

Видим, что на самом деле большой разницы в числе соседей и нет. 

**Можно ли делать только кросс-валидацию (без теста)?**


Нет, нельзя. Кросс-валидация не до конца спасает от подгона параметров модели под выборку, на которой она проводится. Оценка конечного качества модели должно производиться на отложенной тестовой выборке. Если у вас очень мало данных, можно рассмотреть [вложенную кросс-валидацию](https://weina.me/nested-cross-validation/), Речь об этом пойдет позже, в последующих лекциях. Но даже в этом случае придется анализировать поведение модели, чтобы показать, что она учит что-то разумное. Кстати, вложенную кросс-валидацию можно использовать, чтобы просто получить более устойчивую оценку поведения модели на тесте.

## Алгоритм кросс валидации

##Оценка результата кросс-валидации

##Типичные ошибки при кросс-валидации

## Кросс-валидация для научных исследований - на что обратить внимание. 

## GridSearch

##RandomGridSearch

# Линейная регрессия

# Линейный классификатор

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/use_knn_to_comare_imgs_to_find_similar.jpg" width="800">

Давайте подумаем, как избавиться от проблемы со скоростью K-Nearest Neighbors. Реализуя метод ближайшего соседа, мы сравнивали одно изображение со всеми, при этом мы предполагали, что изображения из одного класса будут чем-то похожи друг на друга, и поэтому разница будет меньше. Метод как-то работал. 

Чтобы ускорить этот процесс, мы можем взять весь блок изображений (справа), который относится, например, к машинам и усредним. Таким образом, получим некоторый шаблон для класса “автомобиль”. Скорее всего, работать такой подход будет не слишком здорово, но зато вместо тысяч изображений (в данном случае 50000) появится одно. Возможно, это поможет сильно сэкономить время.

## Переход к сравнению с шаблоном

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/one_template_for_each_class_for_classification.jpg" width="450">

Идея в следующем: вместо того, чтобы сравнивать каждое изображение со всеми остальными по очереди, будем сравнивать изображения с шаблоном для каждого класса. Их будет всего 10 для данного датасета. Этот подход позволит радикально увеличить скорость. 

Проверим, что получится из этой идеи. 


Cкачиваем CIFAR10

In [ ]:
from torchvision.datasets import CIFAR10
import numpy as np

dataset_train = CIFAR10("content", train=True,  download=True)
dataset_test = CIFAR10("content", train=False,  download=True)

X_train = dataset_train.data
Y_train = np.array(dataset_train.targets)

X_test = dataset_test.data
Y_test = np.array(dataset_test.targets)

print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_train shape : {X_test.shape}, Y_test shape: {Y_test.shape}")

Cоздадим шаблоны посчитав среднее значение пикселя по всем изображениям одного класса.


In [ ]:
templates = []  

for i in range(len(dataset_train.classes)):
    imgs = X_train[Y_train == i] # select images by class mask
    mn = np.mean(imgs, 0 )  
    templates.append(mn.astype(int)) # convert to int for display

Визуализируем полученные шаблоны:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20.0, 2.0))

def show_templates(templates, labels):
    for i, template in enumerate(templates):
        plt.subplot(1, len(labels), i + 1)
        plt.title(labels[i])
        plt.axis("off")
        plt.imshow(template)

show_templates(templates, dataset_train.classes)  

Теперь у нас есть 10 шаблонов, с которыми мы будем сравнивать изображение и на основании этого делать предсказание. На этих шаблонах можно увидеть очертания объектов.

Напишем классификатор, который будет сравнивать изображение с шаблоном, который генерируется во время обучения.

In [ ]:
class TemplateBasedClassifier():
    def __init__(self, labels):
        self.templates = []          
        self.labels = labels

    def fit(self, X_train, Y_train):
        self.templates = []
        for label_num in range(len(self.labels)):  
          imgs = X_train[Y_train == label_num] # select images by class mask
          mn = np.mean(imgs, 0 )  
          self.templates.append(mn)  

    def forward(self, x):
        # compute distance score
        distances = np.sum(np.abs(self.templates - x), axis=(1,2,3))  
        return np.argmin(distances)  # return minimum score index

Рассмотрим сравнение на конкретном примере с кошкой. Построим изображения картинки, шаблона, а затем посчитаем расстояние L1 между ними. Чем желтее цвет - тем больше изображение похоже на шаблон

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig,ax = plt.subplots(ncols=3, figsize=(10,5))

car_class_num = 1
imgs = X_train[Y_train == car_class_num] # images class 3(cat)

img = imgs[1] # some image with car
template = templates[car_class_num] 
residual = np.mean(np.abs(img-template), axis = -1) 

# Code for display
ax[0].imshow(img)
ax[1].imshow(template)
r_plot = ax[2].imshow(residual, cmap='inferno_r')

divider = make_axes_locatable(ax[2])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(r_plot, cax=cax, orientation='vertical', label='L1')

ax[0].set_title('Image')
ax[1].set_title('Template')
ax[2].set_title('D = Image - Template')

for a in ax:
    a.axis('off')

In [ ]:
model = TemplateBasedClassifier(dataset_train.classes)  
model.fit(X_train, Y_train)  

Сделаем предсказание и замерим время

In [ ]:
import time
from sklearn.metrics import accuracy_score

def validate(model, X_test, Y_test):
    y_pred = []  # num of correct predictions
    for i, img in enumerate(X_test):  # for all images
        index = model.forward(img)  # predict class
        y_pred.append(index)
    return accuracy_score(Y_test, y_pred)

start = time.perf_counter()  
accuracy= validate(model, X_test, Y_test)  
tm = time.perf_counter() - start 
print(
    "\nAccuracy {:.2f} Train {:d} /test {:d} in {:.1f} sec. speed {:.2f} samples per second.".format(
        accuracy, len(X_train), len(X_test), tm, len(X_test) / tm
    )
)

Если бы мы обучали KNN на таком количестве данных, это заняло бы у нас четверть часа.



##  Переход к весам
----
Умножение вместо вычитания - для чего?


<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/compute_scalar_product_with_compare_with_template.jpg" width="450">

В сравнении с шаблонами, мы вычитали нашу картинку из шаблона и таким образом смотрели, насколько они похожи (L1). Теперь давайте сделаем следующий, пока ничем не обоснованный, ход: оставим всё, как было, но заменим вычитание умножением. Логика в том, что не все пиксели одинаково важны. Вероятно, если на изображении совпадут какие-то пиксели, которые отвечают, например, за глаза кошки, это будет намного важнее, чем фон, который может быть точно таким же у собаки. Здесь будут какие-то важные особенности, которым можно придать больший вес.

Если мы распишем в виде формул наложение шаблона на картинку таким образом с умножением, то получается, что мы скалярно перемножаем два вектора. Подробнее про [скалярное произведение](https://ru.wikipedia.org/wiki/Скалярное_произведение) векторов. Для того, чтобы получить вектор из изображения размерностью 32х32х3, достаточно "выпрямить" его, получим вектор размерностью 1х3072.


Опять рассмотрим сравнение на конкретном примере с кошкой. Построим изображения картинки, шаблона, а затем посчитаем расстояние между ними, используя наш новый метод перемножения. Чем желтее цвет - тем больше изображение похоже на шаблон

In [ ]:
fig,ax = plt.subplots(ncols=3, figsize=(10,5))

indexes = np.where(Y_train == 3)  
mask = np.zeros(len(Y_train), dtype=bool) 
mask[indexes, ] = True  
imgs = X_train[mask]

img = imgs[0].reshape(3, 32, 32).transpose(1, 2, 0).astype(int)
template = templates[3].reshape(3, 32, 32).transpose(1, 2, 0).astype(int)
residual = np.mean(np.abs(img*template),-1)

ax[0].imshow(img)
ax[1].imshow(template)
r_plot = ax[2].imshow(residual, cmap='inferno_r')

divider = make_axes_locatable(ax[2])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(r_plot, cax=cax, orientation='vertical', label='Distance')

ax[0].set_title('Image')
ax[1].set_title('Template')
ax[2].set_title('D = Image * Template')

for a in ax:
    a.axis('off')

###  Математическая запись

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/img_to_vector_to_compute_scalar_product.jpg" width="700">



Обозначим входное изображение как $x$, а шаблон для первого из классов как $w_0$.

Элементы пронумеруем подряд 1,2,3 … $n$. То есть развернем матрицу пикселей изображения в вектор. 

Тогда результат сравнения изображения с этим шаблоном будет вычисляться по формуле: $x[0]*w0[0] + x[1]*w0[1] + … x[n-1]*w0[n-1]$ 




<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/scalar_product_ways_to_use.png" width="800">


Эта простая модель лежит в основе практически всех сложных, которые мы будем рассматривать дальше. Внутри мы будем также пользоваться скалярным произведением.

В дальнейшем мы будем проходить сверточные сети, они работают очень похоже:
мы тоже накладываем шаблон на некоторую матрицу и перемножаем элементы, затем складываем. Единственное отличие – обычно ядро свертки меньше, чем размер самого изображения. 



## Support Vector Machine (метод опорных векторов)

[Отличное видео про SVM от Stat Quest которое все объясняет](https://www.youtube.com/watch?v=efR1C6CvhmE&ab_channel=StatQuestwithJoshStarmer)

### 1D классификация

Рассмотрим одномерный пример. У нас есть данные по массе мышей. Часть из них определена как нормальные, а часть как мыши с ожирением. Чтобы их отделить друг от друга, нам достаточно одного критерия. Мы можем посмотреть на график, и визуально определить предельную массу, после которой мышки будут жирненькими

In [ ]:
import seaborn as sns

def generate_data(total_len=40):
    X = np.hstack([np.random.uniform(14,21, total_len//2), 
                   np.random.uniform(24,33, total_len//2)])
    Y = np.hstack([np.zeros(total_len//2), 
                   np.ones(total_len//2)])
    return X,Y

def plot_data(X, Y, total_len=40, s=50, threshold=None, margin=None):
    ax = sns.scatterplot(x=X, y=np.zeros(len(X)), hue=Y, s=s)
    if threshold:
        ax.axvline(threshold, color='red', ls='dashed')
    if margin:
        for line in margin:
            ax.axvline(line, color='pink', ls='dashed')
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles, ['Normal', 'Obese'])
    ax.set(xlabel='Mass, g');
    return ax

total_len = 40
X, Y = generate_data(total_len=total_len)
ax = plot_data(X, Y, threshold=21.5, total_len=total_len)

Теперь пользуясь нашим простым критерием, попробуем классифицировать каких-то новых мышей

In [ ]:
X_test = np.random.uniform(14,30, 5)

def classify(X, threshold=21.5):
    y = np.zeros_like(X)
    y[X > threshold] = 1
    return y

total_len = 40
threshold = 21.5
X, Y = generate_data(total_len=total_len)
ax = plot_data(X, Y, threshold=threshold, total_len=total_len)
ax = plot_data(X_test, classify(X_test, threshold), total_len=total_len, s=300)

Но что если наши мыши находятся тут?


In [ ]:
X_test = np.array([21.45, 22.5])

total_len = 40
X, Y = generate_data(total_len=total_len)
ax = plot_data(X, Y, total_len=total_len)
ax = plot_data(X_test, classify(X_test), threshold = 21.5, total_len=total_len, s=300)

С точки зрения нашего классификатора - все четко. Больше порогового значения - значит перевес, меньше - значит нормальные. Но с точки зрения здравого смысла, логичнее было бы классифицировать обоих мышей как нормальных, так как они значительно ближе к нормальным, чем к ожиревшим.

Вооружившись этим новым знанием, попробуем классифицировать наших отъевшихся мышек по-умному. Возьмем крайние точки в каждом класстере. И в качестве порогового значения будем использовать среднее между ними

In [ ]:
X, Y = generate_data(total_len=total_len)
normal_limit = X[Y==0].max() # extreme point for 'normal'
obese_limit = X[Y==1].min() # extreme point for 'obese'

threshold = np.mean([normal_limit, obese_limit]) # separated with mean value 

X_test = np.array([21.5, 23])
ax = plot_data(X, Y, total_len=total_len, threshold=threshold, margin=[normal_limit, obese_limit])
ax = plot_data(X_test, classify(X_test, threshold=threshold), total_len=total_len, s=300, threshold=threshold)

Мы можем посчитать, насколько наша мышь близка к тому, чтобы оказаться в другом классе. Такое расстояние называется **margin**. И оно считается как $\mathrm{margin} = |\mathrm{threshold} - \mathrm{observation}|$

In [ ]:
margins = np.abs(X_test - threshold)
print(margins)

Соответственно, если мы посчитаем margins для наших крайних точек `normal_limit` и `obese_limit`, мы найдем самое большое возможное значение margin для нашего классификатора

In [ ]:
margin_0 = np.abs(normal_limit - threshold)
margin_1 = np.abs(obese_limit - threshold)
print(margin_0, margin_1)

Такой классификатор, мы называем **Maximum Margin Classifier**. Он хорошо работает в случае, когда все данные размечены аккуратно. Теперь рассмотрим более реалистичный пример, где что-то пошло не так

In [ ]:
def generate_realistic_data(total_len=40):
    X = np.hstack([np.random.uniform(14,21, total_len//2), np.random.uniform(24,33, total_len//2)])
    Y = np.hstack([np.zeros(total_len//2), np.ones(total_len//2)])
    indx = np.where(X == X[Y==1].min())[0]
    Y[indx] = 0
    s = np.ones_like(X)*50
    s[indx] = 300
    return X,Y,s

total_len = 40
X,Y,s = generate_realistic_data(total_len=total_len)
ax = plot_data(X, Y, total_len=total_len, s=s)

В таком случае, наш **Maximum Margin Classifier** работать не будет. Исходя из этого, мы можем прийти к выводу, что наш классификатор очень чувствителен к выбросам. Давайте подумаем можно ли это как-то исправить?

Например, мы можем разрешить нашему классификатору ошибаться. Если мы будем использовать в качестве порогового значения не самое крайнее, а следующее за ним - мы промажем в классификации конкретно этой странной точки. Но в целом будем правы. Margin определенный таким образом, называется **Soft margin**.

In [ ]:
total_len = 40
X, Y, s = generate_realistic_data(total_len=total_len)

normal_limit = np.sort(X[Y==0])[-2]
obese_limit = np.sort(X[Y==1])[1]

threshold = np.mean([normal_limit, obese_limit])

X_test = np.array([20.5, 25])
ax = plot_data(X, Y, total_len=total_len, threshold=threshold)
ax = plot_data(X_test, classify(X_test, threshold=threshold), total_len=total_len, s=300, threshold=threshold, margin=[normal_limit, obese_limit])

Но почему мы решили взять именно следующее значение? Почему не через 2? Откуда мы знаем что это лучший из возможных вариантов? А ни откуда не знаем. Чтобы узнать какой из margins лучше, нам стоит численно это проверить и посчитать, сколько раз мы ошибемся, если возьмем в качестве порогового значения между каждой парой точек. Для этого мы вновь воспользуемся *кросс-валидацией*.

In [ ]:
total_len = 40
X, Y, s = generate_realistic_data(total_len=total_len)

indxs = []
accuracies = []
thresholds = []

X_test, Y_test = generate_data(total_len=total_len)

for i in range(total_len//2):
    for j in range(total_len//2-1):
        normal_limit = np.sort(X[Y==0])[-i]
        obese_limit = np.sort(X[Y==1])[j]

        threshold = np.mean([normal_limit, obese_limit])
        
        Y_pred = classify(X_test, threshold=threshold)
        accuracy = np.mean(Y_test == Y_pred)
        indxs.append((-i,j))
        accuracies.append(accuracy)
        thresholds.append(threshold)

print(f'Accuracy = {np.max(accuracies)*100}%')
best_index = indxs[np.argmax(accuracies)][1]
print(f'Best indexes', best_index)
best_treshold = thresholds[np.argmax(accuracies)]
print('Best treshhold value %.2f'% best_treshold)

ax = plot_data(X, Y, total_len=total_len, threshold=best_treshold)
ax = plot_data(X_test, Y_test, 
               total_len=total_len, 
               s=200, 
               threshold=best_treshold)

Когда для классификатора используется **Soft Margin** - такой классификатор называют **Soft Margin Classifier** или по другому - **Support Vector Classifier**. По сути это уже SVM. 

### 2D класификация

Теперь рассмотрим пример, где мы измерили не только вес мышей, но и их длину от хвоста до носа. Мы можем вновь применить наш метод Support Vector Classifier, и теперь классы разделяет не одно пороговое значение (по сути, точка), а линия.

In [ ]:
from sklearn.datasets import make_blobs
from sklearn import svm

def generate_2d_data(total_len=40):
    X, Y = make_blobs(n_samples=total_len, centers=2, random_state=42)
    X[:,0] += 10 
    X[:,1] += 20 
    return X, Y

def plot_data(X, Y, total_len=40, s=50, threshold=21.5):
    ax = sns.scatterplot(x=X[:,0], y=X[:,1], hue=Y, s=s)
    handles, labels  =  ax.get_legend_handles_labels()
    ax.legend(handles, ['Normal', 'Obese'])
    ax.set(xlabel='Mass, g', ylabel='Length, cm');
    return ax

total_len = 40
X, Y = generate_2d_data(total_len=total_len)
ax = plot_data(X, Y, total_len=total_len)

# Code for illustration, later we will understand how it works
# fit the model, don't regularize for illustration purposes
clf = svm.SVC(kernel='linear', C=1000)
clf.fit(X, Y)

# plot the decision function
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# create grid to evaluate model
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)

# plot decision boundary and margins
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
           linestyles=['--', '-', '--'])
# plot support vectors
ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100,
           linewidth=1, facecolors='none', edgecolors='k')
plt.show()

### 3D классификация

Если мы добавим еще одно измерение - возраст, мы обнаружим, что наши данные стали трехмерными, а разделяет их теперь не линия, а плоскость

In [ ]:
def generate_3d_data(total_len=40):
    X, Y = make_blobs(n_samples=total_len, centers=2, random_state=42, n_features=3)
    X[:,0] += 10 
    X[:,1] += 20 
    X[:,2] += 10
    return X, Y

def plot_data(X, Y, total_len=40, s=50, threshold=21.5):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(xs=X[:,0], ys=X[:,1], zs=X[:,2], c=Y, s=s, cmap='Set1')
    # plot the decision function
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # create grid to evaluate model
    xx = np.linspace(xlim[0], xlim[1], 30)
    yy = np.linspace(ylim[0], ylim[1], 30)
    YY, XX = np.meshgrid(yy, xx)
    ax.plot_surface(XX, YY, XX*YY*0.2, alpha=0.2)
    handles, labels  =  ax.get_legend_handles_labels()
    ax.legend(handles, ['Normal', 'Obese'])
    ax.set(xlabel='Mass, g', ylabel='Length, cm', zlabel='Age, days');
    return ax

total_len = 40
X, Y = generate_3d_data(total_len=total_len)
ax = plot_data(X, Y, total_len=total_len)


Соответственно, если бы у нас было 4 измерения и больше (например: вес, длинна, возраст, кровяное давление), то многомерная плоскость которая бы разделяла наши классы - называлась бы **гиперплоскость** (рисовать мы ее, конечно же, не будем). Чисто технически, и точка, и линия - тоже гиперплоскости. Но все же гиперплоскостью принято называть то, что нельзя нарисовать на бумаге.

### SVM

Данные не всегда разделяются так хорошо как в случае нашего мышиного датасета. Например, рассмотрим следующее: у нас есть данные по дозировке лекарства и 2 класса - пациенты, которые поправились, и те, которым лучше не стало

In [ ]:
def generate_patients_data(total_len=40):
    X = np.random.uniform(0,50, total_len)
    Y = np.zeros_like(X)
    Y[(X > 15) & (X < 35)] = 1
    return X, Y

def plot_data(X, Y, total_len=40, s=50):
    ax = sns.scatterplot(x=X, y=np.zeros(len(X)), hue=Y, s=s)

    handles, labels  =  ax.get_legend_handles_labels()
    ax.legend(handles, ['Recover', 'Sick'])
    ax.set(xlabel='dose, mg');
    return ax

total_len = 40
X, Y = generate_patients_data(total_len=total_len)
ax = plot_data(X, Y, total_len=total_len)

Соответственно мы не можем найти такое пороговое значение, которое будет разделять наши классы на больных и здоровых, а следовательно, и Support Vector Classifier работать тоже не будет.  Для начала, давайте преобразуем наши данные таким образом, что бы они стали 2-х мерными. В качестве значений по оси Y будем использовать дозу возведенную в квадрат (**доза**$^2$).

In [ ]:
def plot_data(X, Y, total_len=40, s=50):
    ax = sns.scatterplot(x=X[0,:], y=X[1,:], hue=Y, s=s)
    handles, labels  =  ax.get_legend_handles_labels()
    ax.legend(handles, ['Recover', 'Sick'])
    ax.set(xlabel='Dose, mg');
    ax.set(ylabel='Dose$^2$');
    return ax

total_len = 40
X_1, Y = generate_patients_data(total_len=total_len)
X_2 = X_1**2
X = np.vstack([X_1, X_2])

plot_data(X, Y, total_len=40, s=50)
plt.show()

Теперь мы можем вновь использовать Support Vector Classifier для классификации

In [ ]:
plot_data(X, Y, total_len=40, s=50)

x = np.linspace(0,50,50)
xs = [X[0,:][Y==1].min(), X[0,:][Y==1].max()]
ys = [X[1,:][Y==1].min(), X[1,:][Y==1].max()]

# Calculate the coefficients.
coefficients = np.polyfit(xs, ys, 1)

# Let's compute the values of the line...
polynomial = np.poly1d(coefficients)
y_axis = polynomial(x)

# ...and plot the points and the line
plt.plot(x, y_axis, 'r--')
plt.show()

Но тут возникает резонный вопрос - почему мы решили возвести в квадрат? Почему не в куб? Или наоборот не извлечь корень? Как нам решить какое преобразование использовать?

И у нас есть **вторая проблема** - а если перейти надо в пространство очень большой размерности? В этом случае наши данные очень сильно увеличатся в размере.

Комбинация двух проблем дает нам **много радости** - надо перебирать большое число возможных пространств большей размерности



Однако основная фишка Support Vector Machine состоит в том, что внутри он работает на скалярных произведениях. И можно эти скалярные произведения считать, **не переходя в пространство большей размерности**


Для этого SVM использует **Kernel Function**. 

Kernel Function может, например, быть полиномом (**Polynomial Kernel Function**), который имеет параметр $d$ - сколько размерностей выбрать. 

<img src="https://edunet.kea.su/repo/EduNet-content/L02/img_license/svm_kernel_function.png" width="700">

Примеры ядер :

* $k(x_i, x_j) = (<x_i, x_j> + c)^d, с, d \in \mathbb{R}$ - полиномиальное ядро, считает расстояние между объектами в пространстве размерности d

* $k(x_i, x_j) = \frac{1}{z} e^{-\frac{h(x_i, x_j)^2}{h}}$ - радиальная базисная функция RBF


Таким образом, в случае SVM можно легко перебрать много таких пространств на кроссвалидации и выбрать более удобное. 

Более того, SVM может проверять пространства признаков бесконечного размера. Если для такого пространства существует kernel function. Иногда такие пространства оказываются очень удобными для решения задач. Часто используют тот же RBF-пространство, приведенное выше. А оно как раз бесконечно-мерное.

### Регрессия

#### Пример простой линейной регрессии

Теперь ненадолго отвлечемся от SVM и рассмотрим другую задачу. В этой задаче мы будем прогнозировать успеваемость студента, в зависимости от количества часов, которые он учил материал. Это простая задача линейной регрессии, поскольку она включает всего две переменные.

**Регрессия** - это статистический метод, используемый в финансах, инвестировании и других дисциплинах, который пытается определить силу и характер связи между одной зависимой переменной (обычно обозначаемой **Y**) и рядом других переменных (известных как независимые переменные). В задачах регрессии, мы будем пытаться минимизировать ошибку между предсказанием и истинными данными. 

"Регрессия" происходит от слова "регресс", которое, в свою очередь, происходит от латинского "regressus" - возвращаться (к чему-либо). В этом смысле регрессия - это техника, которая позволяет "вернуться назад" от беспорядочных, трудно интерпретируемых данных к более четкой и осмысленной модели.

Загрузим датасет

In [ ]:
!wget https://edunet.kea.su/repo/EduNet-web_dependencies/L02/student_scores.csv
clear_output()

Посмотрим, что там в нем. Видим, что у нас есть два признака - часы и результаты

In [ ]:
import pandas as pd

dataset = pd.read_csv("/content/student_scores.csv")
print(dataset.shape)
dataset.head()

Построим график зависимости одного от другого, а так же отобразим распределения каждой из переменных

In [ ]:
sns.jointplot(data=dataset, x="Scores", y="Hours")
plt.show()

Разделим наши данные на train и test

In [ ]:
from sklearn.model_selection import train_test_split

X = dataset.iloc[:, :-1].values # column Hours
Y = dataset.iloc[:, 1].values # column Score

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

Теперь создадим модель для линейной регрессии. Чтобы не писать с нуля, воспользуемся готовой моделью из библиотеки `sklearn`

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression() 

И обучим ее

In [ ]:
regressor.fit(X_train, Y_train) 

Посмотрим, что получилось

In [ ]:
X_train.shape

In [ ]:
X_points = np.linspace(
    min(X_train), max(X_train), 100
)  # 100 dots at min to max
Y_pred = regressor.predict(X_points)  

plt.figure(figsize=(8, 5))
plt.plot(X_train, Y_train, "o", label="Scores")
plt.plot(X_points, Y_pred, label="y = %.2fx+%.2f" % (regressor.coef_[0], regressor.intercept_ ))
plt.title("Hours vs Percentage", size=15)
plt.xlabel("Hours Studied", size=15)
plt.ylabel("Percentage Score", size=15)
plt.legend()
plt.show()

Теперь сделаем предсказание для тестовой выборки

In [ ]:
Y_pred = regressor.predict(X_test)  

X_points = np.linspace(
    min(X_test), max(X_test), 100
)  
Y_pred = regressor.predict(X_points) 

plt.figure(figsize=(8, 5))
plt.plot(X_test, Y_test, "o", label="Scores")
plt.plot(X_points, Y_pred, label="y = %.2fx+%.2f" % (regressor.coef_[0], regressor.intercept_ ))
plt.title("Hours vs Percentage", size=15)
plt.xlabel("Hours Studied", size=15)
plt.ylabel("Percentage Score", size=15)
plt.legend()
plt.show()

Выглядит не плохо

Посчитаем метрики для наших значений

In [ ]:
from sklearn import metrics

Y_pred = regressor.predict(X_test)

print("Mean Absolute Error: %9.2f" % metrics.mean_absolute_error(Y_test, Y_pred))
print("Mean Squared Error: %10.2f" % metrics.mean_squared_error(Y_test, Y_pred))
print("Root Mean Squared Error: %5.2f" % np.sqrt(metrics.mean_squared_error(Y_test, Y_pred)))

#### Геометрическая интерпретация 

Теперь, когда мы разобрались с тем, что такое регрессия и с чем ее едят, вернемся к нашим картинкам. Как можно применить регрессию для классификации?

Предположим у нас есть только 2 класса. Как можно использовать регрессию для того, чтобы определить относится ли изображение к классу 0 или к классу 1? В упрощенном варианте, задача будет состоять в том, чтобы провести разделяющую плоскость (прямую) между 2-мя классами. Например, мы можем провести прямую через 0. 


<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/regression_for_classification_imgs.png" width="270">





Рассмотрим другую ситуацию, в этом случае, мы не можем просто провести прямую через 0. Но можем отступить от 0 на какое-то расстояние и провести ее там. Вспомним, что уравнение прямой это $y=wx+b$, где $b$ - это смещение (*bias*). Соответственно если b != 0, то прямая через 0 проходить не будет, а будет проходить через значение b.

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/regression_for_classification_add_bias.png" width="270">

[Linear Classification Loss Visualization
](http://vision.stanford.edu/teaching/cs231n-demos/linear-classify/)


Если у нас есть несколько классов (несколько шаблонов), мы можем для каждого из них посчитать уравнение $y_{i} = w_{i}x_{i}+b_{i}$.

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/regression_for_classification_add_bias_add_multiclasses.jpg" width="400">







На картинке нас интересуют 3 класса. Соответственно, мы можем записать систему линейных уравнений:

\begin{aligned}
y_{0} = w_{0}x_{0} + b_{0} \\
y_{1} = w_{1}x_{1} + b_{1} \\
y_{2} = w_{2}x_{2} + b_{2} \\
\end{aligned}

#### Добавление смещения

Мы их можем собрать в матрицу, тогда получится следующее:

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/scalar_product_add_bias.jpg" width="750">

У нас есть матрица коэффициентов, которые мы каким-то образом подобрали, пока ещё не понятно как. Есть вектор $x$, соответствующий изображению. 

Мы умножаем вектор на матрицу, получаем нашу гиперплоскость для четырехмерного пространства в данном случае. Чтобы оно не лежало в 0, мы должны добавить смещение. И мы можем сделать это после, но можно взять и этот вектор смещения (вектор **b**) просто приписать к матрице **W**.

Что будет выходом такой конструкции? Мы умножили матрицу весов на наш вектор, соответствующий изображению, получили некоторый отклик. По этому отклику мы так же, как и при реализации метода ближайшего соседа можем судить: если он больше остальных, то мы предполагаем, что это кошка.


In [ ]:
img = np.array([56, 231, 24, 2])
w_cat = np.array([0.2, -0.5, 0.1, 2.0])
print("Image ", img)
print("Weights ", w_cat)
print("img * w_cat ", img * w_cat)
print("sum ", (img * w_cat).sum())
print("Add bias ", (img * w_cat).sum() + 1.1)

<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/img_to_function_get_scores.jpg" width="600">


Собирая все вместе, получаем какое-то компактное представление, что у нас есть некоторая функция, на вход которой мы подаем изображение, и у нее есть параметры (веса). Пока происходит просто умножение вектора на матрицу, в дальнейшем это может быть что-то более сложное, функция будет представлять какую-то более сложную модель. А на выходе (для классификатора) мы получаем числа, которые интерпретируют уверенность модели в том, что изображение принадлежит к определенному классу.


<img src ="https://edunet.kea.su/repo/EduNet-content/L02/img_license/input_img_scalar_product_add_bias_get_scores.jpg" width="450">



Соответственно, эти коэффициенты, которые являются весами модели, надо каким-то образом подбирать. Но прежде чем подбирать коэффициенты, давайте определимся со следующим: как мы будем понимать, что модель работает хорошо или плохо? Ведь она возвращает достаточно абстрактные числа, которые нужно уметь интерпретировать.


## Понятие линейной регрессии

### Вспоминаем KNN: низкая скорость

### Переход к сравнению с шаблоном

### Переход к скалярному произведению

### Пример линейной регресии

## Метрики регрессии (MSE, MAE, R2)

## Bias-variance tradeoff на примере линейной регрессии и KNN

# Классификация

## Возврат к классификации

## SVM

# Функция потерь

## на примере SVM лосс

# Градиент

## Понятие градиента

## Градиентный спуск

## Большие объемы данных

## Стохастический градиентный спуск

## Регуляризация весов

# Логистическая регрессия

## Логистическая регрессия для бинарной классификации

## Решение с помощью градиентного спуска

## Мультиклассовая классификация (объект может принадлежать к нескольким классам)

## Кросс-энтропия, softmax и логистическая регрессия для мультиклассовой постановки